In [1]:
import numpy as np
import shelve

## Importing 1st stage results

In [7]:
filename = r'bbl.out'

c_list = [899, 2616, 6586, 24763, 30155, 30352] # list of movie theater ids

var_dict = {}

with shelve.open(filename) as ws:
    for c in c_list:
        var_dict[f'{c}_complete'] = ws[f'{c}_completo_add_noise'].astype('float64') # just plug in chosen file

## Objective Function

In [4]:
# takes as args 2-parameter guesses and a tuple with (c_list, stored_results)

def objective(params, *args):
    loss = 0 # loss value to be minimized
    par_array = np.array([params[0],1,params[1]]) # parameter arrays, 2nd parameter value (private shocks) is normalized to 1
    for c in args[0]:
        orig = np.sum(np.dot(args[1][f'{c}_complete'][:,:,0],par_array)) # cval, or original value func
        loss += np.square([max(0, np.dot(var_dict['899_complete'][:,:,i],par_array).sum() - orig) for i in range(
            1,var_dict['899_complete'].shape[2])]).sum() # takes the square of the max of o or the difference between 
                                                            # bias/noise distvals and cval
    return loss

## Minimizing KDE values
Tinkering with guesses and optimization algorithm

In [5]:
from scipy.optimize import minimize

In [11]:
for i in range(10):
    print(minimize(objective, np.array([50*i,50*i]), args=(c_list, var_dict), method='BFGS', options={'maxiter':500}).x)

[-0.00262109 -0.05577977]
[-0.00262109 -0.05577998]
[-0.00262109 -0.05577979]
[-0.00262109 -0.05577979]
[-0.00262109 -0.05577979]
[-0.00262109 -0.05577979]
[-0.00262109 -0.05577981]
[-0.00262109 -0.05577979]
[-0.00262109 -0.05577979]
[-0.00262109 -0.05577977]


In [14]:
results = minimize(objective2, np.array([0,0]), args=(c_list, var_dict), method='BFGS', options={'maxiter':500})

In [21]:
results.x

array([ 0.00204853, -0.10801806])

In [19]:
print(np.square([results.hess_inv[0,0], results.hess_inv[1,1]]))

[4.30532181e-22 4.36964977e-21]


## Minimizing Logit values
Tinkering with guesses and optimization algorithm

In [8]:
minimize(objective, np.array([0,0]), args=(c_list, var_dict), method='Nelder-Mead', options={'maxiter':500})

 final_simplex: (array([[-0.00825531, -0.01817995],
       [-0.00825528, -0.01818136],
       [-0.00825511, -0.01818161]]), array([816543.12822863, 816543.12824683, 816543.12826634]))
           fun: 816543.1282286296
       message: 'Optimization terminated successfully.'
          nfev: 99
           nit: 51
        status: 0
       success: True
             x: array([-0.00825531, -0.01817995])

In [10]:
for i in range(10):
    print(minimize(objective, np.array([50*i,-50*i]), args=(c_list, var_dict), method='BFGS', options={'maxiter':500}).x)

[-0.00825534 -0.01818042]
[-0.00825535 -0.0181804 ]
[-0.00825535 -0.01818039]
[-0.00825535 -0.01818039]
[-0.00825535 -0.01818039]
[-0.00825535 -0.01818037]
[-0.00825534 -0.01818042]
[-0.00825535 -0.01818039]
[-0.00825534 -0.01818042]
[-0.00825535 -0.01818039]


## Results and Hessian Inverse

In [12]:
results = minimize(objective, np.array([0,0]), args=(c_list, var_dict), method='BFGS', options={'maxiter':500})

In [13]:
results.x

array([-0.00825534, -0.01818042])

In [15]:
print(np.sqrt([results.hess_inv[0,0], results.hess_inv[1,1]]))

[2.03327869e-05 1.19361883e-04]
